In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import numpy as np 
import re
import sqlite3
import os
import pdfplumber
from Functions.refactor_dataframe_columns import *

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18992\386034401.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
allFilesinDirectory = os.listdir("../Inputs/")

usefulFiles = []
for k in allFilesinDirectory:
    if k.__contains__("3_1_J"):
        usefulFiles.append(k)
print(usefulFiles)

route = "../Inputs/"
for k in range(len(usefulFiles)):
    usefulFiles[k] = route+usefulFiles[k]
print(usefulFiles)

['DocEconomica_384_3_1_J_2020_12_31_00_00_00_000.pdf', 'DocEconomica_384_3_1_J_2021_12_31_00_00_00_000.pdf', 'DocEconomica_384_3_1_J_2022_12_31_00_00_00_000.pdf']
['../Inputs/DocEconomica_384_3_1_J_2020_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_384_3_1_J_2021_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_384_3_1_J_2022_12_31_00_00_00_000.pdf']


In [4]:
sqliteFile = "../Outputs/datos_ERI2.db"
#sqliteFile = "./datos_ERI2.db"
tableName = "estado_cam_patrimo"

In [5]:
try:
    for document in usefulFiles:
        pdf = pdfplumber.open(document)
        p0 = pdf.pages[0]

        table = p0.extract_table()
        header = table[:6]
        table = table[7:]

        # Helpers in case the file has another format, eg. an extra row
        columnsHeaders = []
        sampleColumnsNumeric = ['301', '302', '303', '30401', '30402']
        
        columnsHeaders = p0.within_bbox((10,185, p0.width, 205))
        columnsHeaders = columnsHeaders.extract_text()
        columnsHeaders = columnsHeaders.split(" ")

        #if the list with the columns, does not containts columns like
        #'301', '302', '303', '30401', cuz the file has another format, so try this
        if not( set(sampleColumnsNumeric).intersection(set(columnsHeaders)) == set(sampleColumnsNumeric) ):
            for k in table:
                if set(sampleColumnsNumeric).intersection(set(k)) == set(sampleColumnsNumeric):
                    columnsHeaders = k

            while None in columnsHeaders:
                columnsHeaders.remove(None)

        columnsHeaders.insert(0,"CODIGO")
        columnsHeaders.insert(0,"CUENTA")
        columnsHeaders.append("TOTAL_PATRIMONIO")


        dataHeader = pd.DataFrame(header) #data
        dataHeader.fillna(np.NaN, inplace=True)
        dataHeader = remove_columns_full_of_Nan(dataHeader, dataHeader.columns.to_list())
        dataHeader.drop([dataHeader.columns.to_list()[0]], axis=1, inplace=True)
        

        dataInfo = pd.DataFrame(table)#daatframeinfo
        dataInfo.replace(re.compile(r'\r'),"", inplace=True)
        dataInfo.replace(re.compile(r'\n'),"", inplace=True)

        flagValue:str = "SALDO AL FINAL DELPERÍODO"
        pos = find_position_of_value_in_column(dataInfo, flagValue)
        rowIndex = dataInfo.index[ dataInfo[dataInfo.columns.to_list()[pos]] == flagValue ].to_list()
        dataInfo = dataInfo.iloc[rowIndex[0]: ]
        dataInfo.reset_index(drop=True, inplace=True)
        
        dataInfo.replace(to_replace=[None], value=np.nan,  inplace=True)

        dataInfo = remove_columns_full_of_Nan(dataInfo, dataInfo.columns.to_list())
        dataInfo.columns = columnsHeaders


        #PÁGINA 2
        p0 = pdf.pages[1]
        tableP1 = p0.extract_tables()

        datosFrameP1 = pd.DataFrame()
        datosContadorRepresentante = pd.DataFrame()
        for k in range(len(tableP1)):
            if k == 0:
                datosFrameP1 = pd.DataFrame(tableP1[k])
            else:
                datosContadorRepresentante = pd.concat([datosContadorRepresentante, pd.DataFrame(tableP1[k])])

        
        datosFrameP1.replace(re.compile(r'\r'),"", inplace=True)
        datosFrameP1.replace(re.compile(r'\n'),"", inplace=True)
        
        datosFrameP1 = datosFrameP1.iloc[4:]

        datosFrameP1.reset_index(drop=True, inplace=True)
        datosContadorRepresentante.reset_index(drop=True, inplace=True)

        flagValue = "REALIZACIÓN DE LARESERVA PORVALUACIÓN DE"
        pos = find_position_of_value_in_column(datosFrameP1, flagValue)

        rowIndex = datosFrameP1.index[ datosFrameP1[datosFrameP1.columns.to_list()[pos]] == flagValue ].to_list()
        datosFrameP1 = datosFrameP1.iloc[rowIndex[0]:]
        datosFrameP1.reset_index(drop=True, inplace=True)


        columnasDFContRepr = datosContadorRepresentante.columns.to_list()
        valoresDFContRepr = datosContadorRepresentante[columnasDFContRepr[0]].to_list()

        if "REPRESENTANTE(S) LEGAL(ES)" not in valoresDFContRepr:
            for k in range(2):
                rowToAdd = {columnasDFContRepr[0]:" "}
                datosContadorRepresentante.loc[-1] = rowToAdd
                datosContadorRepresentante.index = datosContadorRepresentante.index+1
                datosContadorRepresentante.sort_index(inplace=True)
                datosContadorRepresentante.reset_index(drop=True, inplace=True)
            rowToAdd = {columnasDFContRepr[0]:"REPRESENTANTE(S) LEGAL(ES)"}
            datosContadorRepresentante.loc[-1] = rowToAdd
            datosContadorRepresentante.index = datosContadorRepresentante.index+1
            datosContadorRepresentante.sort_index(inplace=True)
            datosContadorRepresentante.reset_index(drop=True, inplace=True)            

        ColumnasContador = ["CONTADOR","IDENT. CONTADOR"]
        ColumnasRepresentante = ["REPRESENTANTE LEGAL","IDENT. REPRESENTANTE LEGAL"]
        datosContadorRepresentante.replace("null", " ", inplace=True)


        datosFrameP1.columns = columnsHeaders

        dataFrameValores = pd.concat([dataInfo, datosFrameP1], ignore_index=True)#DATAFRAME


        datosRepresentante =[]
        datosContador =[]
        valoresDFContRepr=datosContadorRepresentante[columnasDFContRepr[0]].to_list()
        index = 1
        while valoresDFContRepr[index] != "CONTADOR":
            datosRepresentante.append(valoresDFContRepr[index])
            index += 1

        index += 1
        for k in valoresDFContRepr[index:]:
            datosContador.append(k)

        if len(datosContador) > 3:
            datosContador.pop(-1)
            datosContador.pop(-1)

        lenReps = len(datosRepresentante)
        for k in range(lenReps):
            if k%2 == 0 and k != 0:
                ColumnasRepresentante.extend(ColumnasRepresentante)

        lenConts = len(datosContador)
        for k in range(lenConts):
            if k%3 == 0 and k != 0:
                ColumnasContador.extend(ColumnasContador)

        ColumnasRepresentante.extend(ColumnasContador)
        datosRepresentante.extend(datosContador)

        if "" in datosRepresentante:
            datosRepresentante.remove("")

        if len(datosRepresentante) %2 != 0:
            datosRepresentante.pop(-1)


        dfContador = pd.DataFrame([datosRepresentante], columns=ColumnasRepresentante)
        dfContador = rename_duplicate_columns(dfContador)


        transposedHeader = dataHeader.set_index(
            dataHeader.columns.to_list()[0] ).T
        for k in range(len( dfContador.columns.to_list() )):
            transposedHeader.insert(k, dfContador.columns.to_list()[k], dfContador.loc[0,dfContador.columns.to_list()[k]])


        #transposedHeader 
        transposedHeader.columns = [
            remove_accents_and_replace(col) for col in transposedHeader.columns
        ]
        #dataHead = remove_trailing_number_in_column(transposedHeader)

        #dataFrameValores
        dataFrameValores.colums = [
            remove_accents_and_replace(col) for col in dataFrameValores.columns
        ]
        dataFrameValores.columns = [str(val) for val in dataFrameValores.columns.to_list()]
        dataFrameValores = rename_duplicate_columns(dataFrameValores)
        dataFrameValores = remove_trailing_number_in_column(dataFrameValores)
        dataFrameValores.reset_index(drop=True, inplace=True)
        

        columnasTransposedHeader = transposedHeader.columns.to_list()
        
        expAnioIndex = columnasTransposedHeader.index("expediente")
        dataFrameValores.insert(0,"expediente", transposedHeader.loc[dataHeader.columns.to_list()[1], columnasTransposedHeader[expAnioIndex]])
        expAnioIndex = columnasTransposedHeader.index("anio")
        dataFrameValores.insert(0,"anio", transposedHeader.loc[dataHeader.columns.to_list()[1], columnasTransposedHeader[expAnioIndex]])
        

        pdf.close()
        #DB
        conn = sqlite3.connect(sqliteFile)
        cursor = conn.cursor()

        cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{tableName}'")
        table_exists = cursor.fetchone()
        if not table_exists:
            cursor.execute(f"CREATE TABLE {tableName} (id INTEGER PRIMARY KEY AUTOINCREMENT)")


        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='cabecera'")
        table_exists = cursor.fetchone()
        if not table_exists:
            cursor.execute("CREATE TABLE cabecera (id INTEGER PRIMARY KEY)")


        cursor.execute("PRAGMA table_info(cabecera)")
        existing_columns = [column[1] for column in cursor.fetchall()]
        new_columns = [column for column in transposedHeader.columns if column not in existing_columns]

        for column in new_columns:
            cursor.execute(f"ALTER TABLE cabecera ADD COLUMN {column} NULL")

        
        cursor.execute(f"PRAGMA table_info({tableName})")
        existing_columns = [column[1] for column in cursor.fetchall()]
        new_columns = [ column for column in  dataFrameValores.columns if column not in existing_columns]

        for column in new_columns:
            cursor.execute(f"ALTER TABLE {tableName} ADD COLUMN {column} NULL")
        
        conn.commit()
        
        transposedHeader.to_sql("cabecera", conn, if_exists="append", index=False)
        dataFrameValores.to_sql(tableName, conn, if_exists="append", index=False)
        conn.close()


except Exception as e:
    print(f"Error: {e}, {document}")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18992\3342890716.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataHeader.fillna(np.NaN, inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18992\3342890716.py:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataInfo.replace(to_replace=[None], value=np.nan,  inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18992\3342890716.py:163: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing

Columna: 0
Posicion: 0
Columna: 0
Posicion: 0


C:\Users\Usuario\AppData\Local\Temp\ipykernel_18992\3342890716.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataHeader.fillna(np.NaN, inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18992\3342890716.py:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataInfo.replace(to_replace=[None], value=np.nan,  inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18992\3342890716.py:163: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing

Columna: 0
Posicion: 0
Columna: 0
Posicion: 0


C:\Users\Usuario\AppData\Local\Temp\ipykernel_18992\3342890716.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataHeader.fillna(np.NaN, inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18992\3342890716.py:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataInfo.replace(to_replace=[None], value=np.nan,  inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18992\3342890716.py:163: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing

Columna: 0
Posicion: 0
Columna: 0
Posicion: 0
